# Screened Infeasibility UQ: evaluate only

Take the generic DEMO solution, turned into an input file. Remove f-values at iteration vars, and replace their equality constraints with inequalities. Run PROCESS once-through with uncertain inputs, and the response/quantity of interest as the violated constraint residuals, i.e. the infeasibility. This is inexpensive enough to run locally.

Here, the inputs have already been screened using the single-parameter evaluation method to find the most sensitive inputs. This notebook just evaluates the samples and stores them in a database, ready for analysing in in the `infeas_screened_analysis.ipynb` notebook as a later stage.

In [1]:
import easyvvuq as uq
import chaospy as cp
from pathlib import Path
from infeas.decoder import MfileDecoder

## Campaign to capture infeasibility

Using the epistemic uncertain inputs for the entire code, capture the distribution of constraint residuals. To start with, make just 3 inputs uncertain (for running locally).

Begin by defining the parameter bounds and the responses (or quantities of interest).

In [2]:
# Define campaign
WORK_DIR = "campaigns"
Path("campaigns").mkdir(exist_ok=True)
campaign = uq.Campaign(name="example_local", work_dir=WORK_DIR)

# Define parameter space
# Uncertainties from Alex's SA paper

params = {
    "fdene": {
        "type": "float",
        "min": 1.1,
        "max": 1.3,
        "default": 1.2,
    },  # check: not sure if this is right var. I think ok
    "hfact": {"type": "float", "min": 1.0, "max": 1.2, "default": 1.2},
    "coreradius": {"type": "float", "min": 0.45, "max": 0.75, "default": 0.75},  # ok
    "fimp_2": {"type": "float", "min": 0.085, "max": 0.115, "default": 0.1},  # ok
    "fimp_14": {
        "type": "float",
        "min": 1.0e-5,
        "max": 1.0e-4,
        "default": 1.0e-5,
    },  # ok
    "psepbqarmax": {"type": "float", "min": 8.7, "max": 9.7, "default": 9.0},  # ok
    "flhthresh": {"type": "float", "min": 0.85, "max": 1.15, "default": 1.15},  # ok
    "cboot": {
        "type": "float",
        "min": 0.95,
        "max": 1.05,
        "default": 1.0,
    },  # ok
    "peakfactrad": {"type": "float", "min": 2.0, "max": 3.5, "default": 3.33},  # ok
    "kappa": {"type": "float", "min": 1.8, "max": 1.9, "default": 1.848},  # ok
    "etaech": {"type": "float", "min": 0.3, "max": 0.5, "default": 0.4},  # ok
    "feffcd": {"type": "float", "min": 0.5, "max": 5.0, "default": 1.0},  # ok
    "etath": {"type": "float", "min": 0.36, "max": 0.4, "default": 0.375},  # ok
    "etaiso": {"type": "float", "min": 0.75, "max": 0.95, "default": 0.9},  # ok
    "boundl_18": {
        "type": "float",
        "min": 3.25,
        "max": 3.75,
        "default": 3.25,
    },  # q^95_min, ok
    "pinjalw": {"type": "float", "min": 51.0, "max": 61.0, "default": 61.0},  # ok
    "alstroh": {"type": "float", "min": 6.0e8, "max": 7.2e8, "default": 6.6e8},  # ok
    "sig_tf_wp_max": {
        "type": "float",
        "min": 5.2e8,
        "max": 6.4e8,
        "default": 6.4e8,
    },  # ok, but might need sig_tf_case_max to be the same too
    "aspect": {"type": "float", "min": 3.0, "max": 3.2, "default": 3.1},
    "boundu_2": {
        "type": "float",
        "min": 11.0,
        "max": 12.0,
        "default": 12.0,
    },  # B_T^max, ok
    "triang": {"type": "float", "min": 0.4, "max": 0.6, "default": 0.5},  # ok
    "vary_param": {
        "type": "string",
        "default": "",
    },  # param being changed: used for analysis only
    "startupratio": {"type": "float", "min": 0.0, "max": 10.0, "default": 1.0},
}

# QoIs
# Violated constraint residuals
qois = [
    "norm_objf",
    "eq_con001",
    "eq_con002",
    "eq_con011",
    "ineq_con005",
    "ineq_con008",
    "ineq_con009",
    "ineq_con013",
    "ineq_con015",
    "ineq_con030",
    "ineq_con016",
    "ineq_con024",
    "ineq_con025",
    "ineq_con026",
    "ineq_con027",
    "ineq_con033",
    "ineq_con034",
    "ineq_con035",
    "ineq_con036",
    "ineq_con060",
    "ineq_con062",
    "ineq_con065",
    "ineq_con072",
    "ineq_con079",
    "ineq_con081",
    "ineq_con068",
    "ineq_con031",
    "ineq_con032",
    "rms_vio_constr_res",
]

Next, define a custom decoder to handle Process's mfile output.

In [3]:
# Create encoder and decoder
encoder = uq.encoders.GenericEncoder(
    template_fname="demo_sol_max_net_elec_no_f_IN.template", target_filename="IN.DAT"
)

encoder = uq.encoders.GenericEncoder(
    template_fname="cost_conf.template", target_filename="cost_conf.toml",
)

decoder = MfileDecoder(target_filename="MFILE.DAT", output_columns=qois)

cmd = "step_cost_models -i IN.DAT -c cost_conf.toml"
actions = uq.actions.local_execute(encoder, cmd, decoder)

# cmd = "process -i IN.DAT"
# actions = uq.actions.local_execute(encoder, cmd, decoder)

# Add the app
campaign.add_app(name="feasibility", params=params, actions=actions)

In [4]:
# Create PCE sampler
# Vary 3 uncertain inputs
vary = {
    # "fdene": cp.Uniform(
    #     1.1,
    #     1.3,
    # ),
    # "hfact": cp.Uniform(
    #     1.0,
    #     1.2,
    # ),
    # "coreradius": cp.Uniform(
    #     0.45,
    #     0.75,
    # ),
    # "fimp_2": cp.Uniform(
    #     0.085,
    #     0.115,
    # ),
    # "fimp_14": cp.Uniform(
    #     1.0e-5,
    #     1.0e-4,
    # ),
    # "psepbqarmax": cp.Uniform(
    #     8.7,
    #     9.7,
    # ),
    # "flhthresh": cp.Uniform(
    #     0.85,
    #     1.15,
    # ),
    # "cboot": cp.Uniform(
    #     0.95,
    #     1.05,
    # ),
    # "peakfactrad": cp.Uniform(
    #     2.0,
    #     3.5,
    # ),
    # "kappa": cp.Uniform(
    #     1.8,
    #     1.9,
    # ),
    # "etaech": cp.Uniform(
    #     0.3,
    #     0.5,
    # ),
    # "feffcd": cp.Uniform(
    #     0.5,
    #     5.0,
    # ),
    # "etath": cp.Uniform(
    #     0.36,
    #     0.4,
    # ),
    # "etaiso": cp.Uniform(
    #     0.75,
    #     0.95,
    # ),
    # "boundl_18": cp.Uniform(
    #     3.25,
    #     3.75,
    # ),
    # "pinjalw": cp.Uniform(
    #     51.0,
    #     61.0,
    # ),
    # "alstroh": cp.Uniform(
    #     6.0e8,
    #     7.2e8,
    # ),
    # "sig_tf_wp_max": cp.Uniform(
    #     5.2e8,
    #     6.4e8,
    # ),
    # "aspect": cp.Uniform(
    #     3.0,
    #     3.2,
    # ),
    # "boundu_2": cp.Uniform(
    #     11.0,
    #     12.0,
    # ),
    # "triang": cp.Uniform(
    #     0.4,
    #     0.6,
    # ),
    "startupratio": cp.Uniform(
        0.0,
        10.0,
    ),
}
pce_sampler = uq.sampling.PCESampler(vary=vary, polynomial_order=3)

# mc_sampler = uq.sampling.MCSampler(vary=vary, n_mc_samples=10)

# Add pce_sampler to campaign
campaign.set_sampler(pce_sampler)

# Draw samples, execute and collate
campaign.execute().collate(progress_bar=True)
samples = campaign.get_collation_result()

  0%|          | 0/4 [00:00<?, ?it/s]

The IN.DAT file does not contain any obsolete variables.
The IN.DAT file does not contain any obsolete variables.
The IN.DAT file does not contain any obsolete variables.
The IN.DAT file does not contain any obsolete variables.
 The total number of constraints is counted automatically and does not need to be stated in IN.DAT.
 The total number of constraints is counted automatically and does not need to be stated in IN.DAT.
 The total number of constraints is counted automatically and does not need to be stated in IN.DAT.
 The total number of constraints is counted automatically and does not need to be stated in IN.DAT.
 tmargmin_cs and tmargmin should not both be specified in IN.DAT.
 tmargmin_cs has been ignored.
 
 **************************************************************************************************************
 ************************************************** PROCESS ***************************************************
 ************************************** Power Rea

/home/jg6173/easyVVUQ-process/uq-process/lib/python3.10/site-packages/step_cost_models/cost_model_2.py:780: RuntimeWarning: divide by zero encountered in scalar divide
  cv.coecap = 1.0e9 * self.anncap / kwhpy
/home/jg6173/easyVVUQ-process/uq-process/lib/python3.10/site-packages/step_cost_models/cost_model_2.py:798: RuntimeWarning: invalid value encountered in scalar divide
  self.coefwbl = 1.0e9 * self.annfwbl / kwhpy
/home/jg6173/easyVVUQ-process/uq-process/lib/python3.10/site-packages/step_cost_models/cost_model_2.py:813: RuntimeWarning: invalid value encountered in scalar divide
  self.coediv = 1.0e9 * self.anndiv / kwhpy
/home/jg6173/easyVVUQ-process/uq-process/lib/python3.10/site-packages/step_cost_models/cost_model_2.py:852: RuntimeWarning: invalid value encountered in scalar divide
  self.coecdr = 1.0e9 * self.anncdr / kwhpy
/home/jg6173/easyVVUQ-process/uq-process/lib/python3.10/site-packages/step_cost_models/cost_model_2.py:685: RuntimeWarning: divide by zero encountered in s

 Warning in routine OCMMNT :
triang: 0.4 - 0.6; physics_variables.kappa: 1.5 - 2.0;   plascur: 10 - 20 MA, physics_variables.rmajor: 7 - 11 m;
This is longer than 110 columns.
triang: 0.4 - 0.6; physics_variables.kappa: 1.5 - 2.0;   plascur: 10 - 20 MA, physics_variables.rmajor: 7 - 11 m;
rminor: 2 - 3.5 m; tesep: 0 - 0.5 keV; normalised_total_beta: 2 - 3
physics_variables.normalised_total_beta = array(3.25538143)
 Warning in routine OCMMNT :
Triple product = Vol-average electron density x Vol-average         & electron temperature x Energy confinement time:
This is longer than 110 columns.
 Warning in routine OCMMNT :
triang: 0.4 - 0.6; physics_variables.kappa: 1.5 - 2.0;   plascur: 10 - 20 MA, physics_variables.rmajor: 7 - 11 m;
This is longer than 110 columns.
triang: 0.4 - 0.6; physics_variables.kappa: 1.5 - 2.0;   plascur: 10 - 20 MA, physics_variables.rmajor: 7 - 11 m;
rminor: 2 - 3.5 m; tesep: 0 - 0.5 keV; normalised_total_beta: 2 - 3
physics_variables.normalised_total_beta = ar

100%|██████████| 4/4 [00:10<00:00,  2.55s/it]
